In [1]:
# rading the data from csv file and creating a df

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Agg_trial").getOrCreate()



24/08/29 10:14:48 WARN Utils: Your hostname, manu-pc resolves to a loopback address: 127.0.1.1; using 192.168.157.41 instead (on interface wlp58s0)
24/08/29 10:14:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/29 10:14:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.csv("../data/employees.csv", inferSchema=True, header=True)
df.show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|employee_id|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|          NULL|       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|          NULL|       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|          NULL|       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|          NULL|       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|          NULL|       201|           20|


24/08/29 10:15:02 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
rename_column = {
        'DEPARTMENT_ID':'department_id', 
        'FIRST_NAME':'first_name', 
        'LAST_NAME':'last_name', 
        'EMAIL':'email', 
        'HIRE_DATE':'joining_date', 
        'JOB_ID':'job_id', 
        'SALARY':'salary', 
        'COMMISSION_PCT':'commission_pct', 
        'MANAGER_ID':'manager_id'
        }


for old_column, new_column in rename_column.items():
    df = df.withColumnRenamed(old_column,new_column)
df.show()

## otherwise df.withColumnRenamed("SALARY",'salary) can also be used multiple times

+-----------+----------+---------+--------+------------+------------+----------+------+--------------+----------+-------------+
|employee_id|first_name|last_name|   email|PHONE_NUMBER|joining_date|    job_id|salary|commission_pct|manager_id|department_id|
+-----------+----------+---------+--------+------------+------------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|   21-JUN-07|  SH_CLERK|  2600|          NULL|       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|   13-JAN-08|  SH_CLERK|  2600|          NULL|       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|   17-SEP-03|   AD_ASST|  4400|          NULL|       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|   17-FEB-04|    MK_MAN| 13000|          NULL|       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|   17-AUG-05|    MK_REP|  6000|          NULL|  

# Aggregation


## Salary per department 

In [6]:
from pyspark.sql.functions import sum
# aggregate arround the department
df.groupby('department_id').agg(sum("SALARY").alias("total_salary")).show()

+-------------+------------+
|department_id|total_salary|
+-------------+------------+
|           20|       19000|
|           40|        6500|
|          100|       51608|
|           10|        4400|
|           50|       85600|
|           70|       10000|
|           90|       58000|
|           60|       28800|
|          110|       20308|
|           30|       24900|
+-------------+------------+



## Aggregation using SQL

In [8]:
df.createOrReplaceTempView("employees")

query = """
SELECT department_id, sum(salary) as total_salary from employees
GROUP BY department_id
"""

spark.sql(query).show()

+-------------+------------+
|department_id|total_salary|
+-------------+------------+
|           20|       19000|
|           40|        6500|
|          100|       51608|
|           10|        4400|
|           50|       85600|
|           70|       10000|
|           90|       58000|
|           60|       28800|
|          110|       20308|
|           30|       24900|
+-------------+------------+



In [11]:
# Implementing multiple aggregatiion

from pyspark.sql.functions import avg
df.groupBy('department_id').agg(sum('salary'),avg('salary')).show()

+-------------+-----------+------------------+
|department_id|sum(salary)|       avg(salary)|
+-------------+-----------+------------------+
|           20|      19000|            9500.0|
|           40|       6500|            6500.0|
|          100|      51608| 8601.333333333334|
|           10|       4400|            4400.0|
|           50|      85600|3721.7391304347825|
|           70|      10000|           10000.0|
|           90|      58000|19333.333333333332|
|           60|      28800|            5760.0|
|          110|      20308|           10154.0|
|           30|      24900|            4150.0|
+-------------+-----------+------------------+



## Multiple aggregation

In [15]:
from pyspark.sql.functions import round

df.groupBy('department_id').agg(sum('salary'),round(avg('salary'),2)).show()

+-------------+-----------+---------------------+
|department_id|sum(salary)|round(avg(salary), 2)|
+-------------+-----------+---------------------+
|           20|      19000|               9500.0|
|           40|       6500|               6500.0|
|          100|      51608|              8601.33|
|           10|       4400|               4400.0|
|           50|      85600|              3721.74|
|           70|      10000|              10000.0|
|           90|      58000|             19333.33|
|           60|      28800|               5760.0|
|          110|      20308|              10154.0|
|           30|      24900|               4150.0|
+-------------+-----------+---------------------+



In [10]:
from pyspark.sql.functions import avg,sum
df.groupBy('department_id').agg({'salary':'sum'}).show()



+-------------+-----------+
|department_id|sum(salary)|
+-------------+-----------+
|           20|      19000|
|           40|       6500|
|          100|      51608|
|           10|       4400|
|           50|      85600|
|           70|      10000|
|           90|      58000|
|           60|      28800|
|          110|      20308|
|           30|      24900|
+-------------+-----------+



## Aggregation with constraints

In [19]:

df.filter(df['department_id'].isin([10,20])).show()

+-----------+----------+---------+--------+------------+------------+-------+------+--------------+----------+-------------+
|employee_id|first_name|last_name|   email|PHONE_NUMBER|joining_date| job_id|salary|commission_pct|manager_id|department_id|
+-----------+----------+---------+--------+------------+------------+-------+------+--------------+----------+-------------+
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|   17-SEP-03|AD_ASST|  4400|          NULL|       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|   17-FEB-04| MK_MAN| 13000|          NULL|       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|   17-AUG-05| MK_REP|  6000|          NULL|       201|           20|
+-----------+----------+---------+--------+------------+------------+-------+------+--------------+----------+-------------+



In [22]:
# Another way of filtering
from pyspark.sql.functions import col
filterred_df = df.filter(col("department_id").isin([10,20]))

In [24]:
filterred_df.groupBy("department_id").agg(sum('salary'), avg('salary')).show()

+-------------+-----------+-----------+
|department_id|sum(salary)|avg(salary)|
+-------------+-----------+-----------+
|           20|      19000|     9500.0|
|           10|       4400|     4400.0|
+-------------+-----------+-----------+



In [25]:
filterred_df.groupBy("department_id").agg(sum('salary'), avg('salary')).orderBy('department_id', ascending=True).show()


+-------------+-----------+-----------+
|department_id|sum(salary)|avg(salary)|
+-------------+-----------+-----------+
|           10|       4400|     4400.0|
|           20|      19000|     9500.0|
+-------------+-----------+-----------+



## aggregation using window function
DEpartment wise total salary

In [36]:
from pyspark.sql.window import Window

window_spec = Window.partitionBy('department_id')

df_result = df.withColumn('total_salary', sum('salary').over(window_spec))\
    .withColumn('avgsalary', round(avg('salary').over(window_spec),2))

df_result.show()

+-----------+----------+-----------+--------+------------+------------+--------+------+--------------+----------+-------------+------------+---------+
|employee_id|first_name|  last_name|   email|PHONE_NUMBER|joining_date|  job_id|salary|commission_pct|manager_id|department_id|total_salary|avgsalary|
+-----------+----------+-----------+--------+------------+------------+--------+------+--------------+----------+-------------+------------+---------+
|        200|  Jennifer|     Whalen| JWHALEN|515.123.4444|   17-SEP-03| AD_ASST|  4400|          NULL|       101|           10|        4400|   4400.0|
|        201|   Michael|  Hartstein|MHARTSTE|515.123.5555|   17-FEB-04|  MK_MAN| 13000|          NULL|       100|           20|       19000|   9500.0|
|        202|       Pat|        Fay|    PFAY|603.123.6666|   17-AUG-05|  MK_REP|  6000|          NULL|       201|           20|       19000|   9500.0|
|        114|       Den|   Raphaely|DRAPHEAL|515.127.4561|   07-DEC-02|  PU_MAN| 11000|       

In [39]:
df_result.orderBy("department_id").show()


+-----------+----------+-----------+--------+------------+------------+--------+------+--------------+----------+-------------+------------+---------+
|employee_id|first_name|  last_name|   email|PHONE_NUMBER|joining_date|  job_id|salary|commission_pct|manager_id|department_id|total_salary|avgsalary|
+-----------+----------+-----------+--------+------------+------------+--------+------+--------------+----------+-------------+------------+---------+
|        200|  Jennifer|     Whalen| JWHALEN|515.123.4444|   17-SEP-03| AD_ASST|  4400|          NULL|       101|           10|        4400|   4400.0|
|        201|   Michael|  Hartstein|MHARTSTE|515.123.5555|   17-FEB-04|  MK_MAN| 13000|          NULL|       100|           20|       19000|   9500.0|
|        202|       Pat|        Fay|    PFAY|603.123.6666|   17-AUG-05|  MK_REP|  6000|          NULL|       201|           20|       19000|   9500.0|
|        114|       Den|   Raphaely|DRAPHEAL|515.127.4561|   07-DEC-02|  PU_MAN| 11000|       

In [24]:
# Combined code
from pyspark.sql.window import Window
from pyspark.sql.functions import round

win_spec = Window.partitionBy("department_id")


df.withColumn('total_salary',sum('salary').over(win_spec))\
    .withColumn('avg_salary',round(avg('salary').over(win_spec),2))\
    .orderBy('department_id',ascending=False).show()

+-----------+-----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+------------+----------+
|employee_id| FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|total_salary|avg_salary|
+-----------+-----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+------------+----------+
|        205|    Shelley|  Higgins|SHIGGINS|515.123.8080|07-JUN-02|    AC_MGR| 12008|          NULL|       101|          110|       20308|   10154.0|
|        206|    William|    Gietz|  WGIETZ|515.123.8181|07-JUN-02|AC_ACCOUNT|  8300|          NULL|       205|          110|       20308|   10154.0|
|        108|      Nancy|Greenberg|NGREENBE|515.124.4569|17-AUG-02|    FI_MGR| 12008|          NULL|       101|          100|       51608|   8601.33|
|        109|     Daniel|   Faviet| DFAVIET|515.124.4169|16-AUG-02|FI_ACCOUNT|  9000|          NULL|

# Catalyst Optimizer

catalyst optimization is the way of optimizing the large transformation so the it could be cost and time efficient by following these steps
1. filter the data
2. prune the data apply the broadcast joins if one data set is smaller
3. logical execution

In [ ]:
df1 = spark.read.parquet("data1.parquet")
df2 = spark.read.parquet("data2.parquet")

result = df1.filter(df1["age"] > 30) \
            .join(df2, df1["id"] == df2["id"]) \
            .groupBy(df1["city"]) \
            .agg({"salary": "avg"})


In [ ]:
df1 = spark.

In [ ]:
# Take input from the stdin
x=input();
y=input();

# Add two numbers
z=int(x)+int(y);

# Display the sum in the console.
print ('Sum of x+y =',z)

# Interview scenario based

Increment to the hr department only other will have same salary

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

# Initialize Spark session
spark = SparkSession.builder.appName("Salary_Hike").getOrCreate()

# Sample data
data = [("John", "HR", 3000),
        ("Jane", "Finance", 4000),
        ("Sam", "HR", 3500),
        ("Max", "IT", 4500)]

# Create DataFrame
columns = ["name", "dept", "salary"]
df = spark.createDataFrame(data, columns)

# Apply CASE-like logic using when and otherwise
df_with_hike = df.select(
    col("name"),
    when(col("dept") == "HR", col("salary") + col("salary") * 20 / 100)
    .otherwise(col("salary")).alias("hiked_salary")
)

# Show result
df_with_hike.show()


24/09/06 10:30:41 WARN Utils: Your hostname, manu-pc resolves to a loopback address: 127.0.1.1; using 192.168.80.41 instead (on interface wlp58s0)
24/09/06 10:30:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/06 10:30:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+----+------------+
|name|hiked_salary|
+----+------------+
|John|      3600.0|
|Jane|      4000.0|
| Sam|      4200.0|
| Max|      4500.0|
+----+------------+



24/09/06 10:31:00 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
df.select("name", when(col("dept")=="HR", col("salary")+col("salary")*20/100 ).otherwise(col('salary')).alias('hiked') ).show()

+----+------+
|name| hiked|
+----+------+
|John|3600.0|
|Jane|4000.0|
| Sam|4200.0|
| Max|4500.0|
+----+------+



find out the max rated employee from the employee table


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize a Spark session
spark = SparkSession.builder.appName("MaxRatedEmployeeByDepartment").getOrCreate()

# Sample employee data (assuming this is your DataFrame structure)
data = [
    (1, "HR", 4.5),
    (2, "Finance", 3.8),
    (3, "Engineering", 4.9),
    (4, "Engineering", 4.2),
    (5, "HR", 4.7),
    (6, "Finance", 4.1),
    (7, "HR", 4.3),
    (8, "Engineering", 5.0),
    (9, "Finance", 3.6),
    (10, "HR", 4.0)
]
columns = ["employee_id", "department", "rating"]

# Create a DataFrame with the sample data
df = spark.createDataFrame(data, columns)

# Sample 30 rows from the dataset (if the dataset is large, in this case we only have 10 rows)
sample_df = df.sample(withReplacement=False, fraction=1.0).limit(30)

# Group by department and find the employee with the maximum rating in each department
max_rated_df = sample_df.groupBy("department") \
    .agg({"rating": "max"}) \
    .withColumnRenamed("max(rating)", "max_rating")




In [11]:
sample_df.groupBy("department").max("rating").show()

+-----------+-----------+
| department|max(rating)|
+-----------+-----------+
|         HR|        4.7|
|    Finance|        4.1|
|Engineering|        5.0|
+-----------+-----------+



In [25]:
# using window function
from pyspark.sql.functions import max

from pyspark.sql.window import Window

win_spec = Window.partitionBy("department")

sample_df.withColumn('hgh_rating', max('rating').over(win_spec)).show()

+-----------+-----------+------+----------+
|employee_id| department|rating|hgh_rating|
+-----------+-----------+------+----------+
|          3|Engineering|   4.9|       5.0|
|          4|Engineering|   4.2|       5.0|
|          8|Engineering|   5.0|       5.0|
|          2|    Finance|   3.8|       4.1|
|          6|    Finance|   4.1|       4.1|
|          9|    Finance|   3.6|       4.1|
|          1|         HR|   4.5|       4.7|
|          5|         HR|   4.7|       4.7|
|          7|         HR|   4.3|       4.7|
|         10|         HR|   4.0|       4.7|
+-----------+-----------+------+----------+



In [4]:
sample_df.show()

+-----------+-----------+------+
|employee_id| department|rating|
+-----------+-----------+------+
|          1|         HR|   4.5|
|          2|    Finance|   3.8|
|          3|Engineering|   4.9|
|          4|Engineering|   4.2|
|          5|         HR|   4.7|
|          6|    Finance|   4.1|
|          7|         HR|   4.3|
|          8|Engineering|   5.0|
|          9|    Finance|   3.6|
|         10|         HR|   4.0|
+-----------+-----------+------+



In [5]:
max_rated_df.show()

+-----------+----------+
| department|max_rating|
+-----------+----------+
|         HR|       4.7|
|    Finance|       4.1|
|Engineering|       5.0|
+-----------+----------+



In [21]:
# Join with the original DataFrame to get employee details for the max-rated employees
result_df = max_rated_df.alias('m') \
    .join(sample_df.alias('s'), 
          (col('s.rating') == col('m.max_rating')) & (col('s.department') == col('m.department')), 
          how='inner') \
    .select(col('s.employee_id'), col('s.department'), col('s.rating'))

# Show the result
result_df.show()

+-----------+-----------+------+
|employee_id| department|rating|
+-----------+-----------+------+
|          5|         HR|   4.7|
|          6|    Finance|   4.1|
|          8|Engineering|   5.0|
+-----------+-----------+------+



In [23]:
max_rated_df.alias('m').join(sample_df.alias('s'), (col('s.rating') == col("m.max_rating"))\
                                                    & (col('s.department') == col("m.department")),how='inner')\
                                                    .select(col('s.employee_id'), col('s.department'), col('s.rating')).show()

+-----------+-----------+------+
|employee_id| department|rating|
+-----------+-----------+------+
|          5|         HR|   4.7|
|          6|    Finance|   4.1|
|          8|Engineering|   5.0|
+-----------+-----------+------+

